In [1]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import json
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

import yaml
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/parser/embeddings/fasttext-sbwc.3.6.e20.vec'

In [2]:
cantidad = 700000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [3]:
path_to_json = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_seccionado'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)  # for me this prints ['foo.json']
jsons = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        jsons.append(json.load(json_file))
        


In [76]:
jsons[0]['experiencia']

'persona proactiva , ordenar y alto sentir responsabilidad . \n experiencia laboral \n adecco peru \n cargo : gerente desarrollo \n fecha : 01/04/2019 a fecho . \n funciones : \n responsable manejar cartero top cliente . \n responsable cumplimiento objetivo venta líneo negociar , garantizar \n rentabilidad y nivel servicio \n desarrollo convenio diverso institución . \n implementación canal postventa cliente extranjero \n enfoque negociar cliente extranjero y desarrollar servicio a medir \n cliente . \n adecco peru \n cargo : gerente sucursal . \n fecha : 01/01/2017 a 31/03/2019 \n funciones : \n responsable gestión integral sucursal \n responsable cumplimiento objetivo venta líneo negociar , garantizar \n rentabilidad y nivel servicio \n desarrollo y ejecución plan negocio y venta , kpis y estrategia comercial \n orientar a captación , mantenimiento y retención cliente \n cargo : ejecutivo comercial . \n fecha : 25/03/2015 31/12/2016 \n funciones : \n ejecutar estrategia comercial ori

In [80]:
sent = pre_process(jsons[0]['experiencia'])
sent

'persona proactiva ordenar alto sentir responsabilidad experiencia laboral adecco peru cargo gerente desarrollo fecha 01/04/2019 fecho funciones responsable manejar cartero top cliente responsable cumplimiento objetivo venta lineo negociar garantizar rentabilidad nivel servicio desarrollo convenio diverso institucion implementacion canal postventa cliente extranjero enfoque negociar cliente extranjero desarrollar servicio medir cliente adecco peru cargo gerente sucursal fecha 01/01/2017 31/03/2019 funciones responsable gestion integral sucursal responsable cumplimiento objetivo venta lineo negociar garantizar rentabilidad nivel servicio desarrollo ejecucion plan negocio venta kpis estrategia comercial orientar captacion mantenimiento retencion cliente cargo ejecutivo comercial fecha 25/03/2015 31/12/2016 funciones ejecutar estrategia comercial orientar captacion mantenimiento retencion cliente sucursal identificar informacion negocio relevante oportunidad comercial seguimiento propuest

In [81]:
sent_lem = lematizar(sent)
print(sent_lem)

['personar', 'proactiva', 'ordenar', 'alto', 'sentir', 'responsabilidad', 'experiencia', 'laboral', 'adecco', 'peru', 'cargar', 'gerente', 'desarrollar', 'fecho', '01/04/2019', 'fechar', 'funcionar', 'responsable', 'manejar', 'cartero', 'top', 'cliente', 'responsable', 'cumplimiento', 'objetivar', 'venta', 'linear', 'negociar', 'garantizar', 'rentabilidad', 'nivel', 'servicio', 'desarrollar', 'convenio', 'diverso', 'institucion', 'implementacion', 'canal', 'postventa', 'cliente', 'extranjero', 'enfocar', 'negociar', 'cliente', 'extranjero', 'desarrollar', 'servicio', 'medir', 'cliente', 'adecco', 'peru', 'cargar', 'gerente', 'sucursal', 'fecho', '01/01/2017', '31/03/2019', 'funcionar', 'responsable', 'gestion', 'integral', 'sucursal', 'responsable', 'cumplimiento', 'objetivar', 'venta', 'linear', 'negociar', 'garantizar', 'rentabilidad', 'nivel', 'servicio', 'desarrollar', 'ejecucion', 'plan', 'negociar', 'venta', 'kpis', 'estrategia', 'comercial', 'orientar', 'captacion', 'mantenimien

In [83]:
lematizar('ingles')

['ingle']

In [5]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [6]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.8897964


In [55]:
get_closest(word= 'ingle', n = 3)

(['ingle', 'pantorrilla', 'pierna', 'rodilla'],
 [1, 0.733283281326294, 0.7193220853805542, 0.7045726180076599])

In [61]:
model.similarity('tradición', 'tradicional')

0.6669352

In [9]:
import es_core_news_md
nlp = es_core_news_md.load()
nlp_text = nlp('educado')
nlp_text[0].lemma_

'educar'

In [21]:
prc_description = '''ingeniería máster postgrado excel desarrollo gestión comercial experiencia manejo clientes
emprendimiento liderar equipos planificar organizar dirigir trabajo presión seguimiento KPI inglés '''

In [22]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 1
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        #print(similar_words[i], word_value[similar_words[i]])
        #print('------------------------------------------------')

In [23]:
word_value.keys()

dict_keys(['ingeniería', 'ingenierías', 'máster', 'másteres', 'postgrado', 'postgrados', 'excel', 'edexcel', 'desarrollo', 'esarrollo', 'gestión', 'gestión0', 'comercial', 'comerciales', 'experiencia', 'experienciar', 'manejo', 'manejar', 'clientes', 'cliente', 'emprendimiento', 'emprendimientos', 'liderar', 'encabezar', 'equipos', 'bequipos', 'planificar', 'planear', 'organizar', 'organizarla', 'dirigir', 'dirigirlo', 'trabajo', 'trabajode', 'presión', 'presiones', 'seguimiento', 'proseguimiento', 'kpi', 'kpix', 'inglés', 'ingles'])

In [32]:
### ahora veamos si resulta el ranking
#frecuencia de término – frecuencia inversa de documento 
#Tf-idf
#Para calcular este ranking es mejor tener las secciones skills y experiencia
#con el fin de calcular esta metrica usando la ocurrencia de las palabras
# Tenemos todos los CV's y una descripción del cargo, a este descripcion del cargo
# tiene N palabras, le buscamos 2 palabras parecidas, generando una descripcion
# de N*2

# Usando esta nuevo set de palabras de descripcion, recorremos todos los cvs contando 
# la ocurrencia de estas palabras en cada documento, y luego se genera un ranking usando Tf-idf
# La pregunta es: ¿Lo haré sobre el documento entero? o ¿Trataré de seccionar y ocupar ciertas secciones?

no_of_cv = len(jsons)
#print(no_of_cv)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in jsons[i]['skills'] or word in jsons[i]['experiencia'] or word in jsons[i]['educación']:
                #print('entre')
                count[word] += 1
        except:
            pass

    idf[word] = math.log((no_of_cv+1)/(1+count[word]))
    #print(idf)

In [24]:
#calculemos usando similitud



def calculo_similitud(word1, array_palabras):
    n_veces = 0
    for word in array_palabras:
        try:
            sim = model.similarity(word, word1)
            #print(word)
            #print(sim)
            if sim > 0.5:
                n_veces += 1
            else:
                continue
        except: #No estaba la palabra
            pass
        
    return n_veces



word1 = 'feliz'
array = ['alegre', 'contento', 'taladro', 'juguete']




n = calculo_similitud(word1, array)
n

2

In [25]:
jsons[0]['skills'].split()

['marketing',
 'y',
 'gestión',
 'comercial',
 '.',
 'profesión',
 ':',
 'ciencias',
 'comunicación',
 '.']

In [84]:
no_of_cv = len(jsons)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        # eliminación de stopwords y quizas lematizacion
        skill_pro = pre_process(jsons[i]['skills']) 
        expe_pro = pre_process(jsons[i]['experiencia'])
        edu_pro = pre_process(jsons[i]['educación'])
        
        if calculo_similitud(word, skill_pro.split()) or calculo_similitud(word, expe_pro.split()) or calculo_similitud(word, edu_pro.split()):
            count[word] += 1


    idf[word] = math.log((no_of_cv+1)/(1+count[word]))

In [85]:
idf

{'ingeniería': 0.028170876966696224,
 'ingenierías': 0.11778303565638346,
 'máster': 0.0,
 'másteres': 0.750305594399894,
 'postgrado': 0.0,
 'postgrados': 0.0,
 'excel': 0.11778303565638346,
 'edexcel': 0.32542240043462795,
 'desarrollo': 0.44802472252696035,
 'esarrollo': 0.5877866649021191,
 'gestión': 0.14953173397096384,
 'gestión0': 3.58351893845611,
 'comercial': 0.05715841383994862,
 'comerciales': 0.05715841383994862,
 'experiencia': 0.05715841383994862,
 'experienciar': 0.0,
 'manejo': 0.49247648509779424,
 'manejar': 0.0870113769896297,
 'clientes': 0.216223108469636,
 'cliente': 0.05715841383994862,
 'emprendimiento': 0.5877866649021191,
 'emprendimientos': 0.8754687373538999,
 'liderar': 0.4054651081081644,
 'encabezar': 0.4054651081081644,
 'equipos': 0.8109302162163288,
 'bequipos': 0.8109302162163288,
 'planificar': 0.028170876966696224,
 'planear': 0.44802472252696035,
 'organizar': 0.11778303565638346,
 'organizarla': 2.1972245773362196,
 'dirigir': 0.1823215567939546

In [86]:

score = {}
for i in range(no_of_cv):
    score[i] = 0
    skill_pro = pre_process(jsons[i]['skills']) 
    expe_pro = pre_process(jsons[i]['experiencia'])
    edu_pro = pre_process(jsons[i]['educación'])
    for word in word_value.keys():
        
        
        
        tf = 1 + calculo_similitud(word, skill_pro.split()) + calculo_similitud(word, expe_pro.split()) + calculo_similitud(word, edu_pro.split())

        score[i] += word_value[word]*tf*idf[word]

print(score)

{0: 43.97371544878192, 1: 52.73172487302716, 2: 35.36620507515521, 3: 41.688982607979504, 4: 39.48633148270859, 5: 38.00915025296925, 6: 39.9777342772082, 7: 37.314091102654686, 8: 45.506216966039005, 9: 45.08451619761369, 10: 45.908539289348234, 11: 34.97730070522194, 12: 36.95168296529557, 13: 32.03832178745007, 14: 36.95168296529557, 15: 43.956014823812765, 16: 34.58002596363044, 17: 43.80869075714755, 18: 38.52989468194172, 19: 39.85822807533704, 20: 31.472878538718096, 21: 31.5324464690994, 22: 45.66416310670039, 23: 29.76748407082591, 24: 43.4906349798848, 25: 35.77484567122839, 26: 31.864255566596782, 27: 44.11202326489909, 28: 38.6379882151012, 29: 62.6011680342485, 30: 51.1153837476831, 31: 52.6805800505039, 32: 30.992867178170425, 33: 30.711236859702854, 34: 39.01069213962133}


In [87]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], jsons[i]['nombre archivo']))
    
sorted_list.sort(reverse = True)


In [88]:
sorted_list

[(62.6011680342485, '1581023751928-CV'),
 (52.73172487302716, '1580920711029-Kevin_SQ'),
 (52.6805800505039, '1580913938357-CV_Gianfranco_Vela_ESP_Ene.2020'),
 (51.1153837476831, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (45.908539289348234, '1582640494849-ANDRES_MENA_CV'),
 (45.66416310670039, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (45.506216966039005,
  '1580231848251-Curriculum_Vitae_-_Jesús_Alonso_Vásquez_Cole'),
 (45.08451619761369, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (44.11202326489909, '1583796411230-Curriculum__fco_marzo'),
 (43.97371544878192, '1582718980347-CV_-_Mario_Espinoza1'),
 (43.956014823812765, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (43.80869075714755, '1580497335052-Jose_Medina_13-1-2020'),
 (43.4906349798848, '1586786838778-CV_Postulaciones'),
 (41.688982607979504, '1580248761709-CV_Sergio_Soto_Valdes'),
 (39.9777342772082, '1583871756395-CVCabanaGarayChristian1_1'),
 (39.85822807533704, '1581690552715-CV_Gustavo_Martinez_Raya'),
 (39.486331

In [30]:
sorted_list

[(56.895917679714415, '1581023751928-CV'),
 (51.02525455315362, '1580913938357-CV_Gianfranco_Vela_ESP_Ene.2020'),
 (47.478072724082836, '1580920711029-Kevin_SQ'),
 (43.6411227517698, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (41.918002592586824, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (41.80588707391354,
  '1580231848251-Curriculum_Vitae_-_Jesús_Alonso_Vásquez_Cole'),
 (40.02808858756349, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (39.5854724475737, '1582640494849-ANDRES_MENA_CV'),
 (39.144885334831415, '1583796411230-Curriculum__fco_marzo'),
 (39.05049701535349, '1586786838778-CV_Postulaciones'),
 (38.882645416156734, '1580497335052-Jose_Medina_13-1-2020'),
 (38.60155212906874, '1582718980347-CV_-_Mario_Espinoza1'),
 (38.18173490360104, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (35.988874430084614, '1581631090164-CV_ROSSANA_ALAYZA_-_FEBRERO_2020'),
 (35.5020855517845, '1582313013581-CV_FRF'),
 (35.161667216938, '1583871756395-CVCabanaGarayChristian1_1'),
 (35.12451481450

In [36]:
#sorted_list

[(170.78559054055165, '1581023751928-CV'),
 (162.3877890384607, '1580913938357-CV_Gianfranco_Vela_ESP_Ene.2020'),
 (161.7624491017208, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (157.49001028683443, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (156.78137892252016,
  '1580231848251-Curriculum_Vitae_-_Jesús_Alonso_Vásquez_Cole'),
 (156.7429043070174, '1580920711029-Kevin_SQ'),
 (155.98963492021545, '1582640494849-ANDRES_MENA_CV'),
 (155.35479610179192, '1574824594718-Ing._Comercial_Paula_Fetis_CV'),
 (155.2949426863895, '1588283816605-CV_Erika_Sotomayor_Ulloa'),
 (153.43234195022262, '1580248761709-CV_Sergio_Soto_Valdes'),
 (152.9068062113014, '1580497335052-Jose_Medina_13-1-2020'),
 (152.80730807457667, '1586987553214-Felipe_Haupt_Ossa_-CV_1'),
 (152.69372132811037, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (152.51284142364216, '1582718980347-CV_-_Mario_Espinoza1'),
 (152.3960728160635, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (151.84337018481472, '1583796411230-Curriculum__fc

In [42]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
import spacy
import es_core_news_sm


def pre_process(corpus):
    corpus = corpus.lower()

    stopset = stopwords.words('spanish') + list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    return corpus

def lematizar(frase):
    nlp = es_core_news_sm.load()
    doc = nlp(frase)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return lemmas



In [52]:
sent = pre_process(prc_description)
sent_lem = lematizar(sent)

In [53]:
sent

'ingenieria master postgrado excel desarrollo gestion comercial experiencia manejo clientes emprendimiento liderar equipos planificar organizar dirigir trabajo presion seguimiento kpi ingles'

In [54]:
sent_lem

['ingenieria',
 'master',
 'postgrado',
 'excel',
 'desarrollar',
 'gestion',
 'comercial',
 'experiencia',
 'manejar',
 'cliente',
 'emprendimiento',
 'liderar',
 'equipo',
 'planificar',
 'organizar',
 'dirigir',
 'trabajar',
 'presion',
 'seguimiento',
 'kpi',
 'ingle']

In [72]:
sent_2 = pre_process('un ingeniero puntual proactivo con experiencia de manejo de equipos, especialista en redes')
sent_lem_2 = lematizar(sent_2)
sent_2

'ingeniero puntual proactivo experiencia manejo equipos especialista redes'

In [73]:
sent_lem_2

['ingeniero',
 'puntual',
 'proactivo',
 'experiencia',
 'manejar',
 'equipo',
 'especialista',
 'redar']

In [75]:
model.similarity('especialista', 'experto')

0.7584601

In [91]:
file = '/home/erwin/Genoma/cv-parser/parser/Descripcion_cargo/descripcion_cargo'
cv_txt = open(file, "r")
print(cv_txt.read())

Ingeniero informático con experiencia en Java y python.
experiencia en pytorch. Persona comprometida y responsable.
Tendrá contacto directo con el cliente por lo que se requiere manejo de inglés.



In [93]:
d = cv_txt.read()

In [96]:
" ".join([x.strip() for x in cv_txt]) 

''

In [99]:
with open(file) as f:
  descripcion = " ".join([x.strip() for x in f]) 

In [100]:
descripcion

'Ingeniero informático con experiencia en Java y python. experiencia en pytorch. Persona comprometida y responsable. Tendrá contacto directo con el cliente por lo que se requiere manejo de inglés.'

In [95]:
cv_txt

<_io.TextIOWrapper name='/home/erwin/Genoma/cv-parser/parser/Descripcion_cargo/descripcion_cargo' mode='r' encoding='UTF-8'>